# Практическое задание курса Light Auto ML. Часть 2 - LAMA

<details>
<summary>Описание задания</summary>

Основная задача - выбрать и решить соревнование с платформы Kaggle.com  (http://kaggle.com/) , используя два подхода:
1. Подготовить базовое решение (бейзлайн) с помощью Light Auto ML (LAMA)
2. Реализовать альтернативное решение без использования LAMA

Требования к выбору соревнования
- Можно выбрать как текущие, так и прошедшие денежные соревнования
- Другие типы соревнований необходимо согласовать с куратором курса
- Нельзя использовать простые соревнования типа Titanic

Цели проекта
- Превзойти результаты бейзлайна на LAMA
- Продемонстрировать качественный код
- Использовать стандартные подходы к организации кода (например, Pipeline)
- Провести качественный EDA
- Предоставить подробное описание и обоснование гипотез

Критерии оценки
1. Анализ целевой переменной (максимум 1 балл)
[0.5] Численный анализ:
Для регрессии: распределение таргета, поиск аномальных значений
Для классификации: распределение количества классов
[0.5] Визуализация статистик:
- Изолированный анализ
- Анализ во временном контексте

2. Анализ признаков (максимум 4 балла)
[0.5] Типизация признаков (числовые, категориальные, временные) и их распределения
[0.5] Выявление аномальных значений
[0.5] Анализ зависимостей между признаками
[0.5] Анализ пропущенных значений
[0.5] Определение важности признаков (корреляции с таргетом)
[1.0] Графическая визуализация минимум 3-х пунктов выше
[0.5] Анализ возможных преобразований и генерации новых признаков

3. Моделирование (максимум 3.5 балла)
[0.25] Обоснование стратегии разделения данных (train-test split)
Особое внимание уделить предотвращению утечки данных
[0.25] LAMA бейзлайн:
- Минимум 2 различные конфигурации
- Выбор лучшего решения
[3.0] Собственное решение (если не удалось побить LLama baseline: 3 x 1.0 балл за различные пайплайны/попытки):
- Выбор модели
- Построение пайплайна (препроцессинг, обработка пропусков, генерация признаков, отбор признаков, финальная модель/ансамбль)
- Оптимизация гиперпараметров

4. Общие требования к коду (максимум 1.5 балла)
[0.5] Чистый код:
- Оформление ноутбука
- Соответствие PEP 8
- Правильное именование переменных и функций
- Документирование функций
[0.5] Качество кода:
- Следование принципам SOLID
- Отсутствие спагетти-кода
- Обработка предупреждений и ошибок
- Логгирование
[0.5] Структура решения:
- Оформление в виде self-contained pipeline
- Использование стандартных инструментов (например, sklearn pipeline)

Итоговая оценка
Максимальный балл: 10
9-10 баллов: оценка 5А
7-8.5 баллов: оценка 4В
5-6.5 баллов: оценка 3D
Менее 5 баллов: требуется пересдача

Ожидания
Работа должна представлять собой мини-исследование с:
1) Проработкой и проверкой гипотез
2) Оценкой результатов
3) Обоснованием выбора пайплайна
4) Документированием процесса исследования

</details>

#### **Импорт нужных библиотек**

In [8]:
import pandas as pd

from sklearn.model_selection import GroupShuffleSplit
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
import numpy as np

## 3. Моделирование

##### **Повторная загрузка данных**

In [14]:
train = pd.read_csv('./data/train.csv')
test  = pd.read_csv('./data/test.csv')
sample = pd.read_csv('./data/sample_submission.csv')

print(f"Размер тренировочной выборки: {train.shape}")
print(f"Размер тестовой выборки: {test.shape}")
print(f"Размер sample выборки: {sample.shape}")

Размер тренировочной выборки: (6036000, 8)
Размер тестовой выборки: (4024000, 7)
Размер sample выборки: (4024000, 2)


#### **ВАРИАНТ 1**
##### **3.1 Разделение данных - Проверка теории** 
- Обоснование стратегии разделения данных (train-test split)
- Особое внимание уделить предотвращению утечки данных


**Особенности данных:**
- Временные ряды с breath_id (циклы дыхания по 80 шагов)
- Риск утечки данных при случайном split

**Стратегия:**
Используем **GroupShuffleSplit** по breath_id - целые циклы попадают либо в train, либо в validation, но не в оба набора.

**Обоснование:**
Если разделить breath_id между train/val, модель увидит часть цикла и будет предсказывать другую часть того же цикла → завышенное качество из-за утечки.

**Разделение:** 80% train / 20% validation

In [15]:
# # Признаки: ТОЛЬКО ОРИГИНАЛЬНЫЕ для baseline
FEATURES = ['R', 'C', 'time_step', 'u_in', 'u_out']
TARGET = 'pressure'

# Разделение по breath_id
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(gss.split(train, groups=train['breath_id']))

X_train = train.iloc[train_idx]
X_val = train.iloc[val_idx]

print(f"Train: {len(X_train)} строк, {X_train['breath_id'].nunique()} циклов")
print(f"Val: {len(X_val)} строк, {X_val['breath_id'].nunique()} циклов")
print(f"Пересечение breath_id: {len(set(X_train['breath_id']) & set(X_val['breath_id']))}")

Train: 4828800 строк, 60360 циклов
Val: 1207200 строк, 15090 циклов
Пересечение breath_id: 0


#### **3.2 LAMA бейзлайн на пяти конфигурациях**
##### **Config_1. LAMA бейзлайн – FAST + split**

In [ ]:
print("LAMA CONFIG 1: FAST (300 sec)")

task = Task('reg', metric='mae')
automl_fast = TabularAutoML(
    task=task,
    timeout=300,
    cpu_limit=-1,
    reader_params={
        'n_jobs': 4,
        'random_state': 42
    }
)

oof_pred_fast = automl_fast.fit_predict(
    X_train[FEATURES + [TARGET]],
    roles={'target': TARGET},
    verbose=1
)

# Предсказание на validation
val_pred_fast = automl_fast.predict(X_val[FEATURES]).data[:, 0]
mae_fast = np.mean(np.abs(X_val[TARGET] - val_pred_fast))
print(f"\nMAE (Config 1): {mae_fast:.4f}")

In [ ]:
# Предсказание на тест
test_pred_fast = automl_fast.predict(test[FEATURES]).data[:, 0]

submission = pd.DataFrame({
    'id': test['id'],
    'pressure': test_pred_fast
})

submission.to_csv('submissions/lama_config1_submission.csv', index=False)
print(f"Submission сохранен: {len(submission)} строк")
print(submission.head())

Submission сохранен: 4024000 строк
   id  pressure
0   1  6.283615
1   2  5.924276
2   3  7.128025
3   4  8.029167
4   5  9.680011


##### **Config_2. LAMA бейзлайн – DEEP + split**

In [ ]:
print("LAMA CONFIG 2: LONG (1800 sec)")

automl_deep = TabularAutoML(
    task=task,
    timeout=1800,
    cpu_limit=-1,
    general_params={
        'use_algos': [['lgb', 'cb']]
    },
    reader_params={
        'n_jobs': 4,
        'random_state': 42
    }
)

oof_pred_deep = automl_deep.fit_predict(
    X_train[FEATURES + [TARGET]],
    roles={'target': TARGET},
    verbose=1
)

val_pred_deep = automl_deep.predict(X_val[FEATURES]).data[:, 0]
mae_deep = np.mean(np.abs(X_val[TARGET] - val_pred_deep))
print(f"\nMAE (Config 2): {mae_deep:.4f}")

LAMA CONFIG 2: LONG (1800 sec)
[17:00:30] Stdout logging level is INFO.
[17:00:30] Task: reg

[17:00:30] Start automl preset with listed constraints:
[17:00:30] - time: 1800.00 seconds
[17:00:30] - CPU: 11 cores
[17:00:30] - memory: 16 GB

[17:00:30] Train data shape: (4828800, 6)

[17:00:34] Layer 1 train process start. Time left 1795.54 secs
[17:02:02] Selector_LightGBM fitting and predicting completed
[17:02:02] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[17:09:11] Fitting Lvl_0_Pipe_0_Mod_0_LightGBM finished. score = -1.9253601835955747
[17:09:11] Lvl_0_Pipe_0_Mod_0_LightGBM fitting and predicting completed
[17:09:11] Start fitting Lvl_0_Pipe_0_Mod_1_CatBoost ...
[17:18:49] Fitting Lvl_0_Pipe_0_Mod_1_CatBoost finished. score = -2.1692255966336558
[17:18:49] Lvl_0_Pipe_0_Mod_1_CatBoost fitting and predicting completed
[17:18:49] Time left 700.62 secs

[17:18:49] Layer 1 training completed.

[17:18:49] Blending: optimization starts with equal weights. Score = -2.0114945
[17:18:50]

In [ ]:
# Предсказание на тест
test_pred_deep = automl_deep.predict(test[FEATURES]).data[:, 0]

submission = pd.DataFrame({
    'id': test['id'],
    'pressure': test_pred_deep
})

submission.to_csv('submissions/lama_config2_submission.csv', index=False)
print(f"Submission сохранен: {len(submission)} строк")
print(submission.head())

Submission сохранен: 4024000 строк
   id  pressure
0   1  6.245288
1   2  5.945842
2   3  7.185876
3   4  8.135557
4   5  9.929556


### Промеждуточные результаты экспериментов - LAMA baseline на 2 конфигурациях

| Конфигурация | Time limit | Алгоритмы (фактически) | Validation MAE | Kaggle Public | Комментарий |
|---|---|---|---|---|---|
| **Config 1 (Fast + split)** | 300 sec | LightGBM | **1.92** | 3.78 | Быстрое обучение, базовый уровень качества |
| **Config 2 (Deep + split)** | 1800 sec | LGBM + CatBoost | 1.85 | **3.74** | Более стабильная модель, чуть лучше Kaggle score |


Не устроило качество, решила эксперементировать дальше.

##### **Config_3. LAMA бейзлайн – FAST**
Попробуем не делить данные с учетом временных рядов индентификатора, протестируем аналогично, но на стандартном train / test

In [ ]:
train = pd.read_csv('./data/train.csv')
test  = pd.read_csv('./data/test.csv')
sample = pd.read_csv('./data/sample_submission.csv')

print(f"Размер тренировочной выборки: {train.shape}")
print(f"Размер тестовой выборки: {test.shape}")
print(f"Размер sample выборки: {sample.shape}")

In [ ]:
FEATURES_BASE = ['R', 'C', 'time_step', 'u_in', 'u_out']
TARGET = 'pressure'

roles = {
    'target': TARGET,
    'drop': ['breath_id']
}

task = Task('reg', metric='mae')

In [ ]:
automl_base = TabularAutoML(
    task=task,
    timeout=300,
    cpu_limit=-1,
    reader_params={
        'n_jobs': 4,
        'random_state': 42
    }
)

oof_base = automl_base.fit_predict(
    train[FEATURES_BASE + [TARGET]],
    roles=roles,
    verbose=2
)

[15:27:24] Stdout logging level is INFO2.
[15:27:24] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[15:27:24] Task: reg

[15:27:24] Start automl preset with listed constraints:
[15:27:24] - time: 300.00 seconds
[15:27:24] - CPU: 11 cores
[15:27:24] - memory: 16 GB

[15:27:24] Train data shape: (6036000, 6)

[15:27:28] Layer 1 train process start. Time left 296.46 secs
[15:27:28] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[15:27:28] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[15:28:13] Time limit exceeded after calculating fold 0

[15:28:13] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -3.849488213242899
[15:28:13] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[15:28:13] Time left 251.02 secs

[15:29:46] Selector_LightGBM fitting and predicting completed
[15:29:47] Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...
[15:29:47] ===== Start working with fold 0 for Lvl_0_Pipe_1_Mod_0_LightG

In [ ]:
test_pred_base = automl_base.predict(test[FEATURES_BASE]).data[:, 0]

submission_base = pd.DataFrame({
    'id': test['id'],
    'pressure': test_pred_base
})

submission_base.to_csv('submissions/lama_config_fast_baseline.csv',index=False)

##### **Config_4. LAMA бейзлайн – DEEP**

In [ ]:
automl_deep = TabularAutoML(
    task=task,
    timeout=1800,
    cpu_limit=-1,
    general_params={
        'use_algos': [['lgb', 'cb']] # LightGBM и CatBoost
    },
    reader_params={
        'n_jobs': 4,
        'random_state': 42
    }
)

oof_base = automl_deep.fit_predict(
    train[FEATURES_BASE + [TARGET]],
    roles=roles,
    verbose=2
)

[15:41:38] Stdout logging level is INFO2.
[15:41:38] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[15:41:38] Task: reg

[15:41:38] Start automl preset with listed constraints:
[15:41:38] - time: 1800.00 seconds
[15:41:38] - CPU: 11 cores
[15:41:38] - memory: 16 GB

[15:41:38] Train data shape: (6036000, 6)

[15:41:41] Layer 1 train process start. Time left 1796.91 secs
[15:43:12] Selector_LightGBM fitting and predicting completed
[15:43:13] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[15:43:13] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[15:44:44] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[15:46:19] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[15:47:51] ===== Start working with fold 3 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[15:49:23] ===== Start working with fold 4 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[15:50:55] Fitting Lvl_0_Pipe_0_Mod_0_LightGBM fin

In [ ]:
test_pred_base = automl_deep.predict(test[FEATURES_BASE]).data[:, 0]

submission_base = pd.DataFrame({
    'id': test['id'],
    'pressure': test_pred_base
})
submission_base.to_csv('submissions/lama_config_deep_baseline.csv',index=False)

##### **Config_3. LAMA бейзлайн – DEEP + FE**
На этапе eda сформировался стек FE признаков, которые полезно было бы добавить. Попробуем обучит LAMA с дополнительными FE:
**1. Lag-признаки**
- `u_in_lag_1/2/3` - предыдущие значения управляющего сигнала
- так как корреляция внутри циклов в 2-3 раза сильнее глобальной

**2. Difference признаки**
- `u_in_diff` - скорость изменения управляющего сигнала
- динамика изменений критична для временных моделей

**3. Cumulative sum**
- `u_in_cumsum` - накопленный объем воздуха
-  физический смысл - интеграл потока

**4. Взаимодействия R×C**
- `R_x_C` - произведение параметров легких
- график показал нелинейное влияние комбинаций

**5. Позиция в цикле**
- `time_position` - порядковый номер шага внутри breath_id (0-79)
- позволяет модели определять фазу дыхания независимо от абсолютного времени

In [ ]:
def create_features(df):
    """
    Создание признаков на основе анализа:
    - Lag features
    - Difference
    - Cumulative sum
    - R×C interaction
    """
    df = df.copy()
    
    for lag in [1, 2, 3]:
        df[f'u_in_lag_{lag}'] = df.groupby('breath_id')['u_in'].shift(lag)

    df['u_in_diff'] = df.groupby('breath_id')['u_in'].diff()
    df['u_in_cumsum'] = df.groupby('breath_id')['u_in'].cumsum()
    df['R_x_C'] = df['R'] * df['C']
    df['time_position'] = df.groupby('breath_id').cumcount()
    
    # Заполнение NaN в lag-признаках нулями
    lag_cols = [f'u_in_lag_{i}' for i in [1, 2, 3]] + ['u_in_diff']
    df[lag_cols] = df[lag_cols].fillna(0)
    
    print(f"Created {len(lag_cols) + 3} new features")
    return df

# Список всех признаков
ORIGINAL_FEATURES = ['R', 'C', 'time_step', 'u_in', 'u_out']
NEW_FEATURES = ['u_in_lag_1', 'u_in_lag_2', 'u_in_lag_3', 
                'u_in_diff', 'u_in_cumsum', 'R_x_C', 'time_position']
ALL_FEATURES = ORIGINAL_FEATURES + NEW_FEATURES

print(f"Всего признаков: {len(ALL_FEATURES)}")
print(f"   Оригинальные: {len(ORIGINAL_FEATURES)}")
print(f"   Новые: {len(NEW_FEATURES)}")
print(f"\nНовые признаки: {NEW_FEATURES}")

Всего признаков: 12
   Оригинальные: 5
   Новые: 7

Новые признаки: ['u_in_lag_1', 'u_in_lag_2', 'u_in_lag_3', 'u_in_diff', 'u_in_cumsum', 'R_x_C', 'time_position']


In [ ]:
train_fe = create_features(train)
test_fe  = create_features(test)

FEATURES = ['R','C','time_step','u_in','u_out','u_in_lag_1','u_in_lag_2','u_in_lag_3',
            'u_in_diff','u_in_cumsum','R_x_C','time_position']
TARGET = 'pressure'

Created 7 new features
Created 7 new features


In [ ]:
automl_deep_fe = TabularAutoML(
    task=task,
    timeout=1800,
    cpu_limit=-1,
    general_params={
        'use_algos': [['lgb', 'cb']] # LightGBM и CatBoost
    },
    reader_params={
        'n_jobs': 4,
        'random_state': 42
    }
)

oof_base = automl_deep_fe.fit_predict(
    train_fe[FEATURES + [TARGET]],
    roles=roles,
    verbose=2
)

[23:05:31] Stdout logging level is INFO2.
[23:05:31] Task: reg

[23:05:31] Start automl preset with listed constraints:
[23:05:31] - time: 1800.00 seconds
[23:05:31] - CPU: 11 cores
[23:05:31] - memory: 16 GB

[23:05:31] Train data shape: (6036000, 13)

[23:05:36] Layer 1 train process start. Time left 1795.03 secs
[23:07:26] Selector_LightGBM fitting and predicting completed
[23:07:28] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[23:07:28] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[23:09:12] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[23:10:58] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[23:12:45] ===== Start working with fold 3 for Lvl_0_Pipe_0_Mod_0_LightGBM =====
[23:14:30] Time limit exceeded after calculating fold 3

[23:14:30] Fitting Lvl_0_Pipe_0_Mod_0_LightGBM finished. score = -0.5233867584469205
[23:14:30] Lvl_0_Pipe_0_Mod_0_LightGBM fitting and predicting completed
[23:14:30] Start fitting L

In [ ]:
test_pred_base = automl_deep_fe.predict(test_fe[FEATURES]).data[:, 0]

submission_base = pd.DataFrame({
    'id': test['id'],
    'pressure': test_pred_base
})
submission_base.to_csv('submissions/lama_config_deep_fe_baseline.csv',index=False)

### Итоговые результаты экспериментов - LAMA baseline на 5 конфигурациях

| Конфигурация | Time limit | Алгоритмы (фактически) |  Kaggle Private Score |Kaggle Public Score |
|---|---|---|---|---|
| **Config 1 (Fast + split)** | 300 sec | LightGBM |  3.7592 |  3.7776 |
| **Config 2 (Deep + split)** | 1800 sec | LGBM + CatBoost | 3.7262 |  3.7450   |
| **Config 3 (Fast)** | 300 sec | LightGBM |  3.7609 |  3.7813  |
| **Config 4 (Deep)** | 1800 sec | LGBM + CatBoost | 3.7292 | 3.7487 |
| **Config 5 (Deep + fe)** | 1800 sec | LGBM + CatBoost | **0.8491** | **0.8480** |

Лучшим результатом среди всех LAMA оказался с добавляением FE. 
Попробуем побить его в custom_solition.ipynb!!!